# Классификация "фейковых" новостей

Задача:
1. 3 способами получить f1-score > 0.91 через sklearn
2. 3 способами получить f1-score > 0.52 через pytorch

Загрузим данные

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests

In [2]:
# URL для загрузки
url = "https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv"

# Загрузка файла
response = requests.get(url)

# Проверка, что запрос выполнен успешно
if response.status_code == 200:
    # Запись содержимого в файл
    with open("Constraint_Train.csv", "wb") as file:
        file.write(response.content)
    print("Файл успешно загружен и сохранён как 'Constraint_Train.csv'.")
else:
    print(f"Не удалось загрузить файл. Статус код: {response.status_code}")

Файл успешно загружен и сохранён как 'Constraint_Train.csv'.


In [3]:
dataset = pd.read_csv("Constraint_Train.csv")

dataset.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


## Методы sklearn

### TfidfVectorizer + SVC

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer
import re
import string


en_stopwords = stopwords.words("english")
morph = MorphAnalyzer()

In [5]:
def preprocess_text(text):
    text = text.lower()
    # text = re.sub(r"[{}]".format(string.punctuation), "", text)
    text = word_tokenize(text, language="english")
    text = [word for word in text if word not in en_stopwords]
    text = [morph.parse(word)[0].normal_form for word in text]

    return " ".join(text)

In [6]:
dataset.tweet = dataset.tweet.apply(preprocess_text)

In [7]:
vectorizer = TfidfVectorizer(max_features=10000)
bow = vectorizer.fit_transform(dataset.tweet)

In [8]:
bow.shape

(6420, 10000)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [10]:
x_train, x_test, y_train, y_test = train_test_split(bow, dataset.label, test_size=0.33, random_state=42)

In [11]:
from sklearn.svm import SVC

In [12]:
model = SVC(random_state=42)

model.fit(x_train, y_train)

SVC(random_state=42)

In [13]:
y_pred = model.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        fake       0.92      0.94      0.93      1004
        real       0.95      0.93      0.94      1115

    accuracy                           0.93      2119
   macro avg       0.93      0.93      0.93      2119
weighted avg       0.93      0.93      0.93      2119



### Word2Vec + RandomForestClassifier

In [14]:
def get_tokens(text):
    text = word_tokenize(text)

    return text

In [15]:
from gensim.models.word2vec import Word2Vec


dataset.tokens = dataset.tweet.apply(get_tokens)

model_tweets = Word2Vec(dataset.tokens, workers=4, vector_size=300, min_count=3, window=5, epochs=15)

C:\Users\Руслан\AppData\Local\Temp\ipykernel_7348\260935528.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  dataset.tokens = dataset.tweet.apply(get_tokens)


In [16]:
model_tweets.init_sims() # Нормировка векторов

C:\Users\Руслан\AppData\Local\Temp\ipykernel_7348\1821373275.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_tweets.init_sims() # Нормировка векторов


In [17]:
# Токенизация текстов через обученную word2vec
def get_text_embedding(text):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word]) # Добавляем вектор слова, если он был при обучении
    
    if len(result):
        result = np.average(result, axis=0) # Среднее всех векторов
    else:
        result = np.zeros(300) # 300 нулей (размер векторов слов), если не было никаких слов из обученного словаря
    return result

In [18]:
features = dataset.tweet.apply(get_text_embedding)

In [19]:
features = np.array(features.tolist())

In [20]:
x_train, x_test, y_train, y_test = train_test_split(features, dataset.label, test_size=0.33, random_state=42)

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
model = RandomForestClassifier(random_state=42)

model.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

In [23]:
y_pred = model.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        fake       0.95      0.91      0.93      1004
        real       0.92      0.95      0.94      1115

    accuracy                           0.93      2119
   macro avg       0.93      0.93      0.93      2119
weighted avg       0.93      0.93      0.93      2119



### FastText и RandomForestClassifier с подобранными параметрами

In [24]:
from gensim.models.fasttext import FastText


model_tweets = FastText(dataset.tokens, workers=4, vector_size=300, min_count=3, window=5, epochs=15)

In [25]:
# Токенизация текстов через обученную fasttext
def get_text_embedding(text):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word]) # Добавляем вектор слова, если он был при обучении
    
    if len(result):
        result = np.average(result, axis=0) # Среднее всех векторов
    else:
        result = np.zeros(300) # 300 нулей (размер векторов слов), если не было никаких слов из обученного словаря
    return result

In [26]:
features = dataset.tweet.apply(get_text_embedding)

In [27]:
features = np.array(features.tolist())

In [28]:
x_train, x_test, y_train, y_test = train_test_split(features, dataset.label, test_size=0.33, random_state=42)

In [29]:
model = RandomForestClassifier(random_state=42)

model.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

In [30]:
y_pred = model.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        fake       0.93      0.89      0.91      1004
        real       0.91      0.94      0.92      1115

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



In [32]:
from sklearn.model_selection import GridSearchCV


model = RandomForestClassifier(random_state=42)

grid_space={'max_depth':[3, 5, 10, None],
              'n_estimators':[50, 100, 200, 300],
              'max_features':[1, 3, 5, 7],
              'min_samples_leaf':[1, 2, 3],
              'min_samples_split':[1, 2, 3]
           }


grid_search = GridSearchCV(model, grid_space, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(x_train, y_train)

# Лучшие параметры
print("Лучшие параметры:", grid_search.best_params_)
# Лучшая оценка
print("Лучшая оценка:", grid_search.best_score_)
# Лучшая модель
best_clf = grid_search.best_estimator_

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


KeyboardInterrupt: 

In [ ]:
best_clf.fit(x_train, y_train)

y_pred = best_clf.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        fake       0.93      0.89      0.91      1004
        real       0.90      0.94      0.92      1115

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



## Методы PyTorch

### Полносвязный слой

In [31]:
labels = (dataset.label == "real").astype(int).to_list() # Переводим метки в числа

In [32]:
token_lists = [word_tokenize(text.lower()) for text in dataset.tweet]
max_len = len(max(token_lists, key=len))

In [33]:
max_len

1014

In [33]:
from collections import Counter


fb = Counter([len(tokens) for tokens in token_lists])

In [34]:
fb.most_common(10)

[(17, 261),
 (19, 243),
 (18, 241),
 (12, 225),
 (21, 224),
 (16, 224),
 (14, 223),
 (13, 222),
 (15, 216),
 (20, 216)]

Брать максимальный размер в 1014 неразумно, лучше взять более реальное число в 200.

In [35]:
max_len = 200

In [36]:
dataset.tokens = dataset.tweet.apply(get_tokens)

model_tweets = Word2Vec(dataset.tokens, workers=4, vector_size=300, min_count=3, window=5, epochs=15)
model_tweets.init_sims()

C:\Users\Руслан\AppData\Local\Temp\ipykernel_7348\2370146477.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_tweets.init_sims()


In [37]:
# Текст -> вектор через обученную word2vec
def get_text_embedding(tokens, max_len):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in model_tweets.wv:
                result.append(model_tweets.wv[word])
            else:
                result.append(np.zeros(300))
        else:
            result.append(np.zeros(300))
    
    if len(result):
        result = np.average(result, axis=0) # Среднее всех векторов
    else:
        result = np.zeros(300) # 300 нулей (размер векторов слов), если не было никаких слов из обученного словаря
    return result

In [39]:
features = [get_text_embedding(text, max_len) for text in tqdm(token_lists)]

100%|██████████| 6420/6420 [00:00<00:00, 7394.04it/s]


In [41]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.33)

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim

In [42]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.out = nn.Linear(300, 1)
    
    def forward(self, x):
        prediction = torch.sigmoid(self.out(x))
        return prediction
    
net = Net()
print(net)

Net(
  (out): Linear(in_features=300, out_features=1, bias=True)
)


In [59]:
in_data = torch.tensor(x_train).float()
targets = torch.tensor(y_train).float()

In [60]:
in_data.shape

torch.Size([4301, 300])

In [45]:
optimizer = optim.Adam(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [88]:
def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]

        optimizer.zero_grad() # отчистка прошлых градиентов
        output = net(batch_x) # подсчёт предсказанных НС значений
        loss = criterion(output.squeeze(), batch_y) # посчёт функции потерь
        loss.backward() # вычисление градиентов
        optimizer.step() # шаг градиентного спуска
    print(loss)

In [64]:
for _ in range(100):
    train_one_epoch(in_data, targets)

100%|██████████| 269/269 [00:00<00:00, 3501.33it/s]


tensor(0.2180, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 269/269 [00:00<00:00, 3421.24it/s]


tensor(0.1517, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 269/269 [00:00<00:00, 4290.97it/s]


...

tensor(0.0577, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 269/269 [00:00<00:00, 4369.34it/s]


tensor(0.0577, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 269/269 [00:00<00:00, 4607.09it/s]

tensor(0.0576, grad_fn=<BinaryCrossEntropyBackward0>)


In [65]:
in_data_test = torch.tensor(x_test).float()
targets_test = torch.tensor(y_test).float()

In [70]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [77]:
result = (output > 0.5).int()

In [79]:
print(classification_report(targets_test, result))

              precision    recall  f1-score   support

         0.0       0.90      0.90      0.90       987
         1.0       0.92      0.92      0.92      1132

    accuracy                           0.91      2119
   macro avg       0.91      0.91      0.91      2119
weighted avg       0.91      0.91      0.91      2119



### LSTM

In [80]:
# Текст -> вектора токенов через обученную word2vec
def get_word_embedding(tokens, max_len):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in model_tweets.wv:
                result.append(model_tweets.wv[word])
            else:
                result.append(np.zeros(300))
        else:
            result.append(np.zeros(300))
    
    return result

In [81]:
features = [get_word_embedding(text, max_len) for text in tqdm(token_lists)]

100%|██████████| 6420/6420 [00:01<00:00, 5125.15it/s]


In [82]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.33)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(300, 200, dropout=0.3)
        self.out = nn.Linear(200, 1)
    
    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        prediction = torch.sigmoid(self.out(longterm))
        return prediction
    

net = Net()
print(net)

Net(
  (lstm): LSTM(300, 200, dropout=0.3)
  (out): Linear(in_features=200, out_features=1, bias=True)
)


c:\Users\Руслан\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


In [100]:
def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]

        optimizer.zero_grad() # отчистка прошлых градиентов
        output = net(batch_x) # подсчёт предсказанных НС значений
        loss = criterion(output.squeeze(), batch_y) # посчёт функции потерь
        loss.backward() # вычисление градиентов
        optimizer.step() # шаг градиентного спуска
    print(loss)

In [101]:
in_data = torch.tensor(x_train).float()
targets = torch.tensor(y_train).float()

In [102]:
in_data.shape

torch.Size([4301, 200, 300])

In [105]:
optimizer = optim.Adam(net.parameters(), lr=0.005)
criterion = nn.BCELoss()

In [106]:
for _ in range(10):
    train_one_epoch(in_data, targets)

100%|██████████| 269/269 [00:30<00:00,  8.70it/s]


tensor(0.0435, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 269/269 [00:30<00:00,  8.69it/s]


tensor(0.0540, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 269/269 [00:30<00:00,  8.72it/s]


...

tensor(0.0016, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 269/269 [00:30<00:00,  8.74it/s]


tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 269/269 [00:30<00:00,  8.68it/s]

tensor(0.0243, grad_fn=<BinaryCrossEntropyBackward0>)


In [108]:
in_data_test = torch.tensor(x_test).float()
targets_test = torch.tensor(y_test).float()

In [109]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [110]:
result = (output > 0.5).int()

In [ ]:
print(classification_report(targets_test, result))

              precision    recall  f1-score   support

         0.0       0.91      0.95      0.93      1009
         1.0       0.96      0.92      0.94      1110

    accuracy                           0.94      2119
   macro avg       0.94      0.94      0.94      2119
weighted avg       0.94      0.94      0.94      2119



### CNN

In [ ]:
class Net(nn.Module):
    def __init__(self, embedding_dim, n_filters, filter_sizes, dropout):
        super(Net, self).__init__()
        # Устанавливаем количество входных каналов на 1, так как мы будем работать с 1D векторами
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=1, out_channels=n_filters, kernel_size=fs, padding=(fs//2))
            for fs in filter_sizes
        ])
        # Полносвязный слой для классификации
        self.fc = nn.Linear(len(filter_sizes) * n_filters, 1)  # Для бинарной классификации
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.unsqueeze(1)  # Добавляем размерность канала: (batch_size, 1, word_vector_len)
        
        conved = [torch.relu(conv(x)) for conv in self.convs]  # Применяем свертки
        pooled = [torch.max(c, dim=2)[0] for c in conved]  # Пулинг по максимальному значению
        cat = self.dropout(torch.cat(pooled, dim=1))  # Объединяем результаты сверток и применяем dropout
        
        prediction = torch.sigmoid(self.fc(cat))  # Получаем предсказание
        return prediction


net = Net(embedding_dim=300, n_filters=100, filter_sizes=[3, 4, 5], dropout=0.3)
print(net)

Net(
  (convs): ModuleList(
    (0): Conv1d(1, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): Conv1d(1, 100, kernel_size=(4,), stride=(1,), padding=(2,))
    (2): Conv1d(1, 100, kernel_size=(5,), stride=(1,), padding=(2,))
  )
  (fc): Linear(in_features=300, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


In [40]:
features = [get_text_embedding(text, max_len) for text in tqdm(token_lists)]

100%|██████████| 6420/6420 [00:00<00:00, 7381.95it/s]


In [41]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.33)

In [42]:
in_data = torch.tensor(x_train).float()
targets = torch.tensor(y_train).float()

C:\Users\Руслан\AppData\Local\Temp\ipykernel_7348\1569360673.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  in_data = torch.tensor(x_train).float()


In [72]:
optimizer = optim.Adam(net.parameters(), lr=0.005)
criterion = nn.BCELoss()

In [ ]:
def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]

        optimizer.zero_grad() # отчистка прошлых градиентов
        output = net(batch_x) # подсчёт предсказанных НС значений
        loss = criterion(output.squeeze(), batch_y) # посчёт функции потерь
        loss.backward() # вычисление градиентов
        optimizer.step() # шаг градиентного спуска
    print(loss)

In [ ]:
in_data.shape

torch.Size([4301, 300])

In [74]:
for _ in range(100):
    train_one_epoch(in_data, targets)

100%|██████████| 269/269 [00:01<00:00, 266.35it/s]


tensor(0.5390, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 269/269 [00:01<00:00, 266.35it/s]


tensor(0.4432, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 269/269 [00:00<00:00, 269.78it/s]


...

tensor(0.2420, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 269/269 [00:00<00:00, 272.12it/s]


tensor(0.2066, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 269/269 [00:00<00:00, 275.58it/s]

tensor(0.3532, grad_fn=<BinaryCrossEntropyBackward0>)


In [63]:
in_data_test = torch.tensor(x_test).float()
targets_test = torch.tensor(y_test).float()

In [75]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [76]:
result = (output > 0.5).int()

In [77]:
print(classification_report(targets_test, result))

              precision    recall  f1-score   support

         0.0       0.84      0.85      0.84      1000
         1.0       0.86      0.85      0.86      1119

    accuracy                           0.85      2119
   macro avg       0.85      0.85      0.85      2119
weighted avg       0.85      0.85      0.85      2119



## Вывод:

Я научился 6 способами классифицировать тексты (3 - sklearn и 3 - PyTorch). С задачей по метрикам я также справился (лучший результат - 93%, худший - 85%). 

Какой вариант я буду выбирать для дальнейшей работы? - однозначно PyTorch сети RNN (скорее всего, с уже предобученными моделями).